In [ ]:
import argparse
import pandas as pd
import requests
from bs4 import BeautifulSoup # To parse html text
import json, codecs

# BASIC FUNCTIONS

In [ ]:
def parse_html(url):
    html_content = requests.get(url).content
    return BeautifulSoup(html_content, 'html.parser')

In [ ]:
def write_add_file(fname, string_tuple):
    with codecs.open(fname, 'ab', encoding='utf-8') as fout:
        for string in string_tuple:
            fout.write(string)
            fout.write('\n')

def write_replace_file(fname, tupe):
    with codecs.open(fname, 'w', encoding='utf-8') as fout:
        for i in tupe:
            fout.write(i)
            fout.write('\n')

def read_file(filename):
    tupe = []
    with codecs.open(filename, 'r', encoding='utf-8') as f:
        content = f.readlines()
    for link in content:
        link = link.split('\n')[0]
        tupe.append(link)
    return tupe

# GET 'categories.json'

In [ ]:
post_url = 'https://tuoitre.vn/'
try:
    post_tree = parse_html(post_url)
except:
    print 'Cannot read ' + post_url

categories = {}

for item in post_tree.find(class_="clearfix fl").find_all('a'):
    if(item['title'] != 'Home' and item['title'] != 'Media'):
        
        categories_2 = {}
        
        if(item['href'].find('https') == -1):
            url = 'https://tuoitre.vn' + item['href']
            tree = parse_html(url)
            for item2 in tree.find(class_="list-bc").find_all('a'):
                categories_2[item2['title']] = item2['href']

            if(len(categories_2) > 0):
                categories[item['title']] = categories_2
            else:
                categories[item['title']] = item['href']
        else:
            categories[item['title']] = item['href']
        
        
with codecs.open('categories.json', 'w', encoding='utf-8') as fp:
    json.dump(categories, fp, ensure_ascii=False)

# create url by time

In [ ]:
from datetime import datetime
from datetime import timedelta
def list_all_day_in_month(url, month, year):
    url_list_in_moth = []
    front_URL = url + "/xem-theo-ngay/"
    str_date = str(year) + '-' + str(month) + '-1'
    date_object = datetime.strptime(str_date, '%Y-%m-%d')
    while(int(date_object.month) == month):
        if (date_object < datetime.now()):
            url_list_in_moth.append(front_URL + date_object.strftime('%#d-%#m-%Y') + '.html')
            date_object += timedelta(days=1)
        else:
            break
    return url_list_in_moth

In [ ]:
first_year = 2008
last_year = 2019

with codecs.open('categories.json', 'r', encoding='utf-8') as fp:
    categories = json.load(fp)
    url_links = []

for title1 in categories:
    
    back = categories[title1]
    
    if (isinstance(back, basestring)):
        href = back
        if(href.find('https') == -1):
            url = 'https://tuoitre.vn' + href.replace(".htm",'')
            for year in range(first_year, last_year):
                    for month in range(1,13):
                        url_links.extend(list_all_day_in_month(url, month, year))
    else:
        categories_lv2 = back
        href_main = ""
        for title2 in categories_lv2:
            
            if(categories_lv2[title2].find('https') == -1):
                href_main = categories_lv2[title2].split('/')[1]
                href2 = categories_lv2[title2].split('/')[2]
                
                url2 = 'https://tuoitre.vn/' + href2.replace(".htm",'')
                for year in range(first_year, last_year):
                        for month in range(1,13):
                            url_links.extend(list_all_day_in_month(url2, month, year))
                        
        if(href_main != ""):
            url_main = 'https://tuoitre.vn/' + href_main
            for year in range(first_year, last_year):
                for month in range(1,13):
                    url_links.extend(list_all_day_in_month(url_main, month, year))

write_replace_file('TEMPT',url_links)
write_replace_file('TEMPT.BAK',url_links)

# get links

In [ ]:
def get_links(url, date): # them do chiu dung
    url_tuple = []

    while True:
        tree = parse_html(url)
        list_news_content = tree.find(class_="list-news-content pdt30")

        if(list_news_content != None):
            break

    for item in list_news_content.find_all(class_="block-right-info fr"):
        url_tuple.append('https://tuoitre.vn' + item.find('a')['href']  + ',' + url.split('/')[3] + ',' + date)
    return url_tuple

In [ ]:
def get_page_links(link):
    links_tuple = []
    
    date = link.split('/')[5].replace('.html', '').replace('-', '/')

    html_tree = parse_html(link)
    mgr = html_tree.find(class_="page_right mgr15")

    if mgr != None:
        mgr15 = mgr.find_all('a')
        for item in mgr15[:len(mgr15) - 1]:
            links_tuple.extend(get_links('https://tuoitre.vn' + item['href'], date))
    else:
        links_tuple.extend(get_links(link, date))
    return links_tuple

In [ ]:
from time import sleep

i = 0

url_links = read_file('TEMPT')
    
for link in url_links:
    
    if(i%100 == 0):
        sleep(10)
        
    write_add_file('LINKS', get_page_links(url_links[i]))
    
    url_links.remove(url_links[i])
    write_replace_file('TEMPT', url_links)

    i += 1
    print "\r%d" %len(url_links),

In [ ]:
189150/4